In [410]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
import torch
model_id = 'hongseongpil/Wav2Vec2.0_zeroth_Ko'
model = Wav2Vec2ForCTC.from_pretrained(model_id,output_attentions=True)
model.eval()
processor = Wav2Vec2Processor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [411]:
import torch
from pydub.effects import normalize
from pydub import AudioSegment 
from pydub.silence import detect_nonsilent
import numpy as np
def getAudioInput(source_path , silentremove = True):
    try:
        audio = AudioSegment.from_file(source_path).set_frame_rate(16000).set_sample_width(2).set_channels(1)
        print(audio.duration_seconds)
        newaudio = AudioSegment.empty()
        if(silentremove):
            result = detect_nonsilent(audio,min_silence_len=200,silence_thresh=-60)
            newaudio = AudioSegment.empty()
            for index in result:
                newaudio += audio[index[0]:index[1]]
            newaudio = normalize(newaudio)
        else:
            newaudio = normalize(audio)
        return newaudio
    except Exception as e:
        print("오류 발생:", e)
def PrintAudioInfo(audio):
    channels = audio.channels
    sample_rate = audio.frame_rate
    print("Channels:", channels)
    print("Sample rate:", sample_rate)
    print("Duration: ", audio.duration_seconds)
    print("Bit depth:", audio.sample_width, "bits") 
    print("len samples:", len(np.array(audio.get_array_of_samples())))
    display(audio)

In [412]:
def GetLogits(audio):
    input = processor(np.array(audio.get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
    with torch.no_grad():
        input_values = torch.tensor(input).unsqueeze(0)
        return model(input_values).logits
    
def DisplayPred(logits , k):
    sorted_tensor, indices = torch.sort(logits, dim=2, descending=True)
    outputs = list()
    for i,v in enumerate(indices[0]):
        outputs.append((processor.tokenizer.convert_ids_to_tokens(v[:k].tolist()),v[:k].tolist()))
    for i,v in enumerate(outputs):
        print(sorted_tensor[0][i][:k],v)

def DisplayResult(logits, audio , showaudio = False):
    predlogits = torch.argmax(logits, dim=-1)[0]
    outputs = processor.decode(predlogits,output_char_offsets=True)
    time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate
    PrintAudioInfo(audio)
    print( [w['char'] for w in outputs['char_offsets']])
    for w in outputs['char_offsets']:
        start_offset = round (w[ "start_offset" ] * time_offset, 2 )
        end_offset = round (w[ "end_offset" ] * time_offset, 2 )
        print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)
        if(showaudio):
            display(audio[start_offset*1000:end_offset*1000])
    


In [436]:
from Tokenize_Kor import decompose_tokens
audio = getAudioInput("weather_00201001.wav" , False)
logits = GetLogits(audio)
predlogits = torch.argmax(logits, dim=-1)[0]
outputs = processor.decode(predlogits,output_char_offsets=True)
origintext = "요즘은 고령자도 가입 가능한거지요?"
origintext = "".join(decompose_tokens(origintext)).replace(" ","")
predtext = outputs['text'].replace(" ","")

print(origintext)
print(predtext)
DisplayResult(logits,audio)

4.2100625


C:\Users\zx288\AppData\Local\Temp\ipykernel_15856\3547003041.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input).unsqueeze(0)


ㅇㅛㅈㅡㅁㅡㄴㄱㅗㄹㅕㅇㅈㅏㄷㅗㄱㅏㅇㅣㅂㄱㅏㄴㅡㅇㅎㅏㄴㄱㅓㅈㅣㅇㅛ
ㅇㅛㅈㅡㅁㅜㄴㄱㅗㄷㅗㅇㅅㅏㄷㅗㅅㅏㅇㅣㅂㄱㅏㄴㅡㅇㅎㅏㄴㄱㅓㅈㅛ
Channels: 1
Sample rate: 16000
Duration:  4.2100625
Bit depth: 2 bits
len samples: 67361


['ㅇ', 'ㅛ', 'ㅈ', 'ㅡ', 'ㅁ', 'ㅜ', 'ㄴ', 'ㄱ', 'ㅗ', 'ㄷ', 'ㅗ', 'ㅇ', 'ㅅ', 'ㅏ', 'ㄷ', 'ㅗ', ' ', 'ㅅ', 'ㅏ', 'ㅇ', 'ㅣ', 'ㅂ', ' ', 'ㄱ', 'ㅏ', 'ㄴ', 'ㅡ', 'ㅇ', ' ', 'ㅎ', 'ㅏ', 'ㄴ', ' ', 'ㄱ', 'ㅓ', 'ㅈ', 'ㅛ']
word :  ㅇ start_offset : 0.18 end_offset : 0.2
word :  ㅛ start_offset : 0.22 end_offset : 0.24
word :  ㅈ start_offset : 0.32 end_offset : 0.34
word :  ㅡ start_offset : 0.34 end_offset : 0.36
word :  ㅁ start_offset : 0.46 end_offset : 0.48
word :  ㅜ start_offset : 0.48 end_offset : 0.5
word :  ㄴ start_offset : 0.52 end_offset : 0.54
word :  ㄱ start_offset : 0.66 end_offset : 0.68
word :  ㅗ start_offset : 0.68 end_offset : 0.7
word :  ㄷ start_offset : 0.76 end_offset : 0.78
word :  ㅗ start_offset : 0.8 end_offset : 0.82
word :  ㅇ start_offset : 0.82 end_offset : 0.84
word :  ㅅ start_offset : 0.96 end_offset : 0.98
word :  ㅏ start_offset : 1.0 end_offset : 1.02
word :  ㄷ start_offset : 1.12 end_offset : 1.14
word :  ㅗ start_offset : 1.14 end_offset : 1.16
word :    start_offset : 1.22 end_offset : 1.26
wor

In [645]:
from Tokenize_Kor import isVowel
def CalPenality(s1,s2 , i , j):
    c1 = s1[i]
    c2 = s2[j]
    penalty = 1
    c1_vowel = isVowel(c1)
    c2_vowel = isVowel(c2)
    if(c1_vowel == c2_vowel):
        penalty = 0.5
        if(not c1_vowel):
            ## 조음 위치가 비슷한 음소는 패널티를 적게 부여하자..
            if((c1 =='ㅈ') and (c2 == 'ㅉ')):
                penalty = 0.3
    else:
        if((c1 == 'ㅇ') or (c2=='ㅇ')):
            penalty = 0.3
    return penalty
def levenshtein_distance_Marix(s1, s2):
    m, n = len(s1), len(s2)
    # DP 테이블 초기화
    dp = np.zeros((m+1, n+1))
    # DP 테이블 첫 번째 행과 열 초기화
    for i in range(1, m+1):
        dp[i][0] = i
    for j in range(1, n+1):
        dp[0][j] = j
    # DP 테이블 채우기
    for i in range(1, m+1):
        for j in range(1, n+1):
            dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
            if s1[i-1] != s2[j-1]:
                dp[i][j]  += CalPenality(s1,s2,i-1,j-1)
    return dp

def Expand_Dist_Matrix(mat , s1, s2):
    m , n  = len(mat), len(mat[0])
    dp = np.pad(mat, ((0, 1), (0, 1)), mode='constant', constant_values=0)
    dp[m][0] = m
    dp[0][n] = n
    for i in range(1, m):
        dp[i][n] = min(dp[i-1][n], dp[i][n-1], dp[i-1][n-1]) 
        if s1[i-1] != s2[n-1]:
            dp[i][n] += CalPenality(s1,s2,i-1,n-1)
    for j in range(1, n):
        dp[m][j] = min(dp[m][j-1], dp[m-1][j], dp[m-1][j-1])
        if s1[m-1] != s2[j-1]:
            dp[m][j] += CalPenality(s1,s2,m-1,j-1)

    dp[m][n] = min(dp[m-1][n-1], dp[m][n-1], dp[m-1][n])     
    if s1[m-1] != s2[n-1]:
        dp[m][n] += CalPenality(s1,s2,m-1,n-1)  
    return dp


def DisplayMatrix(s1, s2):
    dp = levenshtein_distance_Marix(s1,s2)
    # 첫 번째 행 출력
    print("        ", end="")  # 첫 번째 열은 빈칸으로 시작
    for char in s2:
        print(f"{char:^4}", end="")
    print()
    # 나머지 행 출력
    for i in range(len(s1)+1):
        if(i != 0):
            print(s1[i-1], end=" ")  # 각 행의 첫 번째 열에 s1의 문자 출력
        else:
            print("    ", end="")
        for j in range(len(s2)+1):
            print(f"{dp[i][j]:.2f}", end=" ")  # 소수점 이하 2자리까지 출력
        print()

In [664]:
def PhonemeMatching(origintext,predtext, search):
    matching = [] 
    s1_len = len(origintext)
    s2_len = len(predtext)
    s1_index = 0
    s2_index = 0
    isMinIndice = False
    exec = True
    while(exec):
        size = search
        dist_mat = levenshtein_distance_Marix(origintext[s1_index:s1_index+size], predtext[s2_index:s2_index+size])
        min_indices = np.empty(0)
        if( s1_index + size >= s1_len or s2_index + size >= s2_len):
            matching.append([s1_index,s2_index,s1_len,s2_len])
            exec = False
            break
        candi = np.concatenate([dist_mat[size,:size+1],dist_mat[:size,size]])
        min_indices = np.where(candi == np.min(candi))[0]
        isMinIndice = (min_indices.size == 1)
        while (not isMinIndice):
            if( s1_index + size >= s1_len or s2_index + size >= s2_len):
                matching.append([s1_index,s2_index,s1_len,s2_len])
                exec = False
                break
            dist_mat = Expand_Dist_Matrix(dist_mat,origintext[s1_index:s1_index+size+1], predtext[s2_index:s2_index+size+1])
            candi = np.concatenate([dist_mat[size,:size+1],dist_mat[:size,size]])
            min_indices = np.where(candi == np.min(candi))[0]
            isMinIndice = (min_indices.size == 1)
            if(not isMinIndice):
                size +=1
        if(exec):
            min_indices = min_indices.squeeze(0)
            if(min_indices <= size):
                matching.append([s1_index,s2_index,s1_index+size,s2_index+min_indices])  
                s1_index = s1_index+size
                s2_index = s2_index+min_indices
            else:
                matching.append([s1_index,s2_index,s1_index+min_indices,s2_index+size])  
                s1_index = s1_index+min_indices
                s2_index = s2_index+size
    return matching

In [667]:
def inference(s1,s2 , search = 10 , step = 1):
    if((search < step) ):
        return [[s1,s2]]
    matching = PhonemeMatching(s1,s2,search)
    infer = []
    for v in matching:
        result = inference(s1[v[0]:v[2]],s2[v[1]:v[3]],search-step)
        infer.extend(result)
    return infer
infer = inference(decompose_tokens("아아안녕"),decompose_tokens("안녕"))


In [668]:
infer

[[['ㅇ'], ['ㅇ']],
 [['ㅏ', 'ㅇ', 'ㅏ'], ['ㅏ']],
 [['ㅇ'], ['ㄴ']],
 [['ㅏ', 'ㄴ', 'ㄴ'], ['ㄴ']],
 [['ㅕ'], ['ㅕ']],
 [['ㅇ'], ['ㅇ']]]

In [669]:
DisplayMatrix(decompose_tokens("아안녕"),decompose_tokens("안녕"))
dp = levenshtein_distance_Marix(decompose_tokens("아안녕"),decompose_tokens("안녕"))

         ㅇ   ㅏ   ㄴ   ㄴ   ㅕ   ㅇ  
    0.00 1.00 2.00 3.00 4.00 5.00 6.00 
ㅇ 1.00 0.00 0.30 0.80 1.30 1.60 1.60 
ㅏ 2.00 0.30 0.00 1.00 1.80 1.80 1.90 
ㅇ 3.00 0.30 0.30 0.50 1.00 1.30 1.30 
ㅏ 4.00 0.60 0.30 1.30 1.50 1.50 1.60 
ㄴ 5.00 1.10 1.30 0.30 0.30 1.30 1.80 
ㄴ 6.00 1.60 2.10 0.30 0.30 1.30 1.80 
ㅕ 7.00 1.90 2.10 1.30 1.30 0.30 0.60 
ㅇ 8.00 1.90 2.20 1.80 1.80 0.60 0.30 
